<a href="https://colab.research.google.com/github/jaitrasaketh/Neural-Network-from-Scratch/blob/main/neural_networks_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'digit-recognizer:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3004%2F861823%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241014%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241014T124113Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D66a6b376baf713d1dcabbe52063f6eb3f2ade890c948e51bc52f7dede21ef6689be208c704465551e866860faabd4deef94c24eef06ddf2d248dc4f6480d3bc7905cef4bc3578dffeb8972960103d0cf182832104ec334d9e609e1544d62a3ee71f04037e59fa944d2d1d9dc0a494ee562d3c71e301fd5edb174dfe105f19711acac0394f9ebb4460cae0d59bb7f23948bb19bc829923e759fb998823faca07b0bada5841d06560b6eef5fc95a8441200ad8c46aa9f71a0acb0a38c740b5400c7858c0b7575279688a06818365f57b07767900c34384e796d44e7e62d441446fc2db74da2b28cfe9aa356ae963140d1d4aad00ec7dd32800629b8d181e8cb1de'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
data=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
data.head()


In [ ]:
data.shape

In [ ]:
data=np.array(data)

m, n= data.shape


np.random.shuffle(data)



data_valid=data[0:1000].T
Y_valid=data_valid[0]
X_valid=data_valid[1:n]


data_train=data[1000:m].T
Y_train=data_train[0]
X_train=data_train[1:n]

In [ ]:
def init_params():
    W1=np.random.rand(10,784)
    b1=np.random.rand(10,1)
    W2=np.random.rand(10,10)
    b2=np.random.rand(10,1)
    return W1,b1,W2,b2

In [ ]:
def ReLU(Z):
    return np.maximum(0,Z)

def softmax(Z):
    return np.exp(Z)/np.sum(np.exp(Z))

In [ ]:
def forward_propagation(W1,X,b1,W2,b2):
    Z1=W1.dot(X)+b1
    A1=ReLU(Z1)
    Z2=W2.dot(A1)+b2
    A2=softmax(Z2)

    return Z1,A1,Z2,A2



In [ ]:
def one_hot_encoding(Y):
    one_hot_Y=np.zeros((Y.size,Y.max()+1))
    one_hot_Y[np.arange(Y.size),Y]=1
    one_hot_Y=one_hot_Y.T
    return one_hot_Y

In [ ]:
def der_ReLU(Z):
    return Z>0    ## Z>) is a boolean exp. if Z>0 returns 1 which is the derivative when Z>0 , or returns 0 which is the derivative when Z<0

In [ ]:
def backward_propagation(A2,Y,A1,Z1,X,W2):
    dZ2=A2-one_hot_encoding(Y)
    dW2=1/(Y.size)*dZ2.dot(A1.T)
    db2=1/(Y.size)*np.sum(dZ2,2)

    dZ1=W2.T .dot(dZ2) * der_ReLU(Z1)
    dW1=1/(Y.size)*dZ1.dot(X.T)
    db1=1/(Y.size)*np.sum(dZ1)

    return dW1,db1,dW2,db2


In [ ]:
def update_params(W1,b1,W2,b2,alpha,dW1,db1,dW2,db2):

    W1=W1-alpha*dW1
    b1=b1-alpha*db1
    W2=W2-alpha*dW2
    b2=b2-alpha*db2
    return W1,b1,W2,b2

In [ ]:
def get_predictions(A2):
    return np.argmax(A2,0)

def get_accuracy(predictions,Y):
    return np.sum(predictions==Y)/Y.size

In [ ]:
def gradient_descent(X,Y,iterations,alpha):
    W1,b1,W2,b2=init_params()

    for i in range(iterations):
        Z1,A1,Z2,A2=forward_propagation(W1,X,b1,W2,b2)
        dW1,db1,dW2,db2=backward_propagation(A2,Y,A1,Z1,X,W2)
        W1,b1,W2,b2=update_params(W1,b1,W2,b2,alpha,dW1,db1,dW2,db2)
        if(i%100==0):
            print("Iteration :",i)
            print("Accuracy :",get_accuracy(get_predictions(A2),Y))




    return W1,b1,W2,b2

In [ ]:
W1,b1,W2,b2=gradient_descent(X_train,Y_train,1000,0.1)